In [1]:
#This should result in a class doing aggregate work on idle machines.
from pydantic import BaseModel
from automate_dump_load import Automated_load_dump_for_machine, Points_times
import dataloader
from schemas import Trip
from tqdm import tqdm
from datetime import datetime, timedelta
import plotly.graph_objects as go
import os, sys
import plotly.express as px
import pandas as pd
import numpy as np
import ipyleaflet as L

In [2]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [3]:
class Idle_machine(BaseModel):
    machine_id: str | int
    trips: list[Trip]
    list_of_idle_times: list[Points_times]
    total_idle_seconds: float


class Idle_machines(BaseModel):
    list_of_idle_machines: list[Idle_machine] = []


class Aggregated_idle_machines:
    
    def __init__(self, day: str, machine_type: str) -> None:
        print("Initializing...")
        #Initializing Idle_machine
        self.idle_machines = Idle_machines()
        #Need to know machines for day
        trip = dataloader.TripsLoader(day)

        print("Computing idle time for relevant machines...")
        for machine_id in tqdm(trip._machines.keys()):
            if trip._machines[machine_id].machine_type == machine_type:
                with HiddenPrints():
                    temp_automated = Automated_load_dump_for_machine(day, machine_id)
                    temp_automated.find_idle_time()
                temp_total_time_idle_seconds = sum(
                    [
                        (l.times[-1] - l.times[0]).total_seconds()
                        for l in temp_automated.stats.list_of_idle_times
                    ]
                )

                self.idle_machines.list_of_idle_machines.append(Idle_machine(
                        machine_id = temp_automated.machine.machine_id,
                        trips = temp_automated.machine.trips,
                        list_of_idle_times = temp_automated.stats.list_of_idle_times,
                        total_idle_seconds = temp_total_time_idle_seconds)
                    )
        print("Finished!")

    def plot_violin_idle(self):
        dict_of_idle_times_each_machine = {}
        #Plot violin idle time of each machine
        #For each machine, generate alist containing spans in seconds of each idle time
        #Plot together in a violin plot, will let us see individual differences
        for m in self.idle_machines.list_of_idle_machines:
            m.machine_id
            idle_seconds_list = [(p.times[-1]-p.times[0]).total_seconds() for p in m.list_of_idle_times]
            dict_of_idle_times_each_machine[m.machine_id] = idle_seconds_list
        
        df = pd.DataFrame.from_dict(dict_of_idle_times_each_machine, orient='index')
        df = df.transpose()

        fig = px.violin(df)

        # Customize the plot as needed (e.g., labels, title)
        fig.update_layout(
            xaxis_title="Machine ID",
            yaxis_title="Idle time in seconds",
            title="Violin Plot of idle time for machine IDs"
        )

        # Show the plot
        fig.show()

    def aggragated_nb_of_idle_timeline(self):
        
        first_timestamp = self.idle_machines.list_of_idle_machines[0].trips[0].positions[0].timestamp #First machines first timestamp
        last_timestamp = self.idle_machines.list_of_idle_machines[0].trips[-1].positions[-1].timestamp #First machines last timestamp
        for machine in self.idle_machines.list_of_idle_machines:
            if machine.trips[0].positions[0].timestamp < first_timestamp:
                first_timestamp = machine.trips[0].positions[0].timestamp
            if machine.trips[-1].positions[-1].timestamp > last_timestamp:
                last_timestamp  = machine.trips[-1].positions[-1].timestamp
        
        #Create a list of timestamps throughout day
        current_datetime = first_timestamp
        self.datetime_intervals = []

        while current_datetime < last_timestamp:
            self.datetime_intervals.append(current_datetime)
            current_datetime += timedelta(minutes=2) #This could be a parameter
        
        self.nb_of_idle_machines = [0 for i in self.datetime_intervals]
        self.nb_of_machines_in_action = [0 for i in self.datetime_intervals]
        #Now have a list of times, and list of machines
        for i in range(len(self.datetime_intervals)):
            time = self.datetime_intervals[i]
            for m in self.idle_machines.list_of_idle_machines:
                if m.trips[0].positions[0].timestamp < time < m.trips[-1].positions[-1].timestamp:
                   self.nb_of_machines_in_action[i] += 1 
                for it in m.list_of_idle_times:
                    if it.times[0] < time < it.times[-1]:
                        self.nb_of_idle_machines[i] += 1
                        break

    def plot_aggragated_nb_of_idle_timeline(self):

        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x=self.datetime_intervals,
            y=self.nb_of_idle_machines,
            mode='markers+lines',
            name='Machines idle'
        ))

        fig.add_trace(go.Scatter(
            x=self.datetime_intervals,
            y=self.nb_of_machines_in_action,
            mode='markers+lines',
            name='Machines in action'
        ))


        fig.update_layout(
            title='Number of concurrently idle machines',
            xaxis_title='Time',
            yaxis_title='Machines idle',
            xaxis=dict(type='date'),
            yaxis=dict(type='linear'),
        )

        fig.show()
    
    def plot_peak_times(self, threshold: int):

        if not len(self.datetime_intervals) > 0 :
            self.aggragated_nb_of_idle_timeline()

        
        last_val = 0 #Want to avoid plotting similar maps for same idle period
        for i in range(len(self.datetime_intervals)):
            list_of_positions = []
            if self.nb_of_idle_machines[i] > threshold and self.nb_of_idle_machines[i] > last_val:
                last_val = self.nb_of_idle_machines[i]
                #We are at or above threshold. Want to plot position of idle machines
                time = self.datetime_intervals[i]
                print("At: ", time)
                print("Idle machines: ", self.nb_of_idle_machines[i])
                for m in self.idle_machines.list_of_idle_machines:
                    for it in m.list_of_idle_times:
                        if it.times[0] < time < it.times[-1]:
                            list_of_positions.append(it.points[0])#Assuming its not moving a lot during this interval
                            break
        
                # Create a map centered at the mean of all coordinates, with heatmap
                points_center = np.mean(list_of_positions, axis=0)
                m = L.Map(center=(points_center[0], points_center[1]), zoom=10)
                for pos in list_of_positions:
                    load_mark = L.Marker(location=pos)
                    m.add_layer(load_mark)
                # Display the map
                display(m)
            else:
                last_val = 0

In [4]:
day = "04-09-2022"  # MM-DD-YYYY
machine_type = 'Truck'

agg = Aggregated_idle_machines(day, machine_type)

Initializing...
Computing idle time for relevant machines...


 81%|████████  | 17/21 [10:08<01:45, 26.46s/it]

In [ ]:
agg.aggragated_nb_of_idle_timeline()
agg.plot_aggragated_nb_of_idle_timeline()

In [ ]:
agg.plot_peak_times(15)

In [ ]:
len(agg.idle_machines.list_of_idle_machines)

In [ ]:
#Average idle seconds for each idle machine
for i in agg.idle_machines.list_of_idle_machines:
    print(i.total_idle_seconds/len(i.trips))